In [1]:
!pip install -U s3fs fsspec aiobotocore boto3

  Using cached boto3-1.37.38-py3-none-any.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
  Using cached boto3-1.37.37-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.36-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.35-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.34-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.33-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.32-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.31-py3-none-any.whl.metadata (6.7 kB)
INFO: pip is still looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
  Using cached boto3-1.37.30-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.29-py3-none-any.whl.metadata (6.7 kB)
  Using cached boto3-1.37.28-py3-none-any.whl.metadata (6.7 kB)
  Usin

In [2]:
import xarray as xr
xr.__version__

'2025.3.1'

In [3]:
import s3fs

In [4]:
import glob
import os

In [5]:
import virtualizarr as vz
vz.__version__

'1.3.3.dev40+ge92c0ef'

In [6]:
import zarr
zarr.__version__

'3.0.6'

In [7]:
bucket_url = 's3://cworthy/oae-efficiency-atlas/'

In [8]:
single_file_url = bucket_url + 'data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-01.nc'

In [9]:
s3fs_kwargs = {'anon': True, 'endpoint_url': 'https://data.source.coop'}

In [10]:
fs = s3fs.S3FileSystem(**s3fs_kwargs)

In [11]:
single_simulation_path = bucket_url + 'data/experiments/000/01/'

In [12]:
single_simulation_paths = fs.ls(single_simulation_path)
single_simulation_urls = ['s3://' + path for path in single_simulation_paths]
single_simulation_urls[0:10]

['s3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-01.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-02.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-03.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-04.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-05.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-06.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-07.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-08.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.000-1999-01.pop.h.0347-09.nc',
 's3://cworthy/oae-efficiency-atlas/data/experiments/000/01/alk-forcing.0

In [13]:
os.environ['LITHOPS_CONFIG_FILE'] = '/Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/.lithops_config'

In [14]:
# needed to do combine_by_coords, and each has only one value on each dataset
ENSEMBLE_DIMENSION_COORDS = [
    'elapsed_time',
    'polygon_id',
    'injection_date',
]
OTHER_DIMENSION_COORDS = [
    'time_bounds',
    'z_t',
    'z_w',
    'z_t_150m',
    'z_w_bot',
    'z_w_top',
]
# so tiny it takes less memory to store their values than to store references to them
SCALARS = [
    'T0_Kelvin',
    'cp_air',
    'cp_sw',
    'days_in_norm_year',
    'fwflux_factor',
    'grav',
    'heat_to_PW',
    'hflux_factor',
    'latent_heat_fusion',
    'latent_heat_fusion_mks',
    'latent_heat_vapor',
    'mass_to_Sv',
    'momentum_factor',
    'nsurface_t',
    'nsurface_u',
    'ocn_ref_salinity',
    'omega',
    'ppt_to_salt',
    'radius',
    'rho_air',
    'rho_fw',
    'salinity_factor',
    'salt_to_Svppt',
    'salt_to_mmday',
    'salt_to_ppt',
    'sea_ice_salinity',
    'sflux_factor',
    'sound',
    'vonkar',
    'stefan_boltzmann'
]
LOW_DIMENSIONAL_VARS = ENSEMBLE_DIMENSION_COORDS + OTHER_DIMENSION_COORDS+ SCALARS

Only need to run the following two cells if you haven't deployed this runtime yet

In [15]:
!lithops clean -b aws_lambda

2025-04-22 14:55:46,760 [INFO] cli.py:107 -- Cleaning all Lithops information
2025-04-22 14:55:46,760 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-04-22 14:55:46,905 [INFO] aws_s3.py:59 -- S3 client created - Region: us-west-2
2025-04-22 14:55:47,619 [INFO] aws_lambda.py:97 -- AWS Lambda client created - Region: us-west-2
2025-04-22 14:55:47,960 [INFO] aws_lambda.py:313 -- Deleting function: lithops-worker-r2ie-361dev0-a857601c4a
2025-04-22 14:55:48,343 [INFO] utils.py:83 -- Deleting objects from bucket 'lithops-us-west-2-r2ie' and prefix 'lithops.runtimes/aws_lambda'
2025-04-22 14:55:49,823 [INFO] utils.py:93 -- Finished deleting objects, total found: 1
2025-04-22 14:55:49,824 [INFO] utils.py:83 -- Deleting objects from bucket 'lithops-us-west-2-r2ie' and prefix 'lithops.jobs'
2025-04-22 14:55:51,272 [INFO] utils.py:93 -- Finished deleting objects, total found: 1
2025-04-22 14:55:51,277 [INFO] cli.py:141 -- All Lithops temporary data cleaned


In [16]:
!lithops runtime build -b aws_lambda -f /Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/Dockerfile virtualizarr-runtime

2025-04-22 14:55:51,742 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-04-22 14:55:51,742 [DEBUG] config.py:101 -- Loading configuration from /Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/.lithops_config
2025-04-22 14:55:51,750 [DEBUG] config.py:179 -- Loading Serverless backend module: aws_lambda
2025-04-22 14:55:51,781 [DEBUG] aws_lambda.py:53 -- Creating AWS Lambda client
2025-04-22 14:55:51,891 [INFO] aws_lambda.py:97 -- AWS Lambda client created - Region: us-west-2
2025-04-22 14:55:51,891 [INFO] aws_lambda.py:335 -- Building runtime virtualizarr-runtime from /Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/Dockerfile
cmd='/usr/local/bin/docker build --platform=linux/amd64 --provenance=false --no-cache -t virtualizarr-runtime -f /Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/Dockerfile . '
2025-04-22 14:55:51,892 [DEBUG] utils.py:234 -- Creating function handler zip in lithops_lambda.zip
[+] Building 0.0s (0/0)  docker:default
[+] Buildin

In [17]:
!lithops runtime deploy -b aws_lambda virtualizarr-runtime

2025-04-22 15:00:42,683 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-04-22 15:00:42,683 [DEBUG] config.py:101 -- Loading configuration from /Users/tom/Documents/Work/Code/VirtualiZarr/examples/oae/.lithops_config
2025-04-22 15:00:42,696 [DEBUG] config.py:179 -- Loading Serverless backend module: aws_lambda
2025-04-22 15:00:42,762 [DEBUG] config.py:219 -- Loading Storage backend module: aws_s3
2025-04-22 15:00:42,763 [DEBUG] aws_s3.py:36 -- Creating AWS S3 Client
2025-04-22 15:00:42,867 [INFO] aws_s3.py:59 -- S3 client created - Region: us-west-2
2025-04-22 15:00:43,258 [DEBUG] aws_lambda.py:53 -- Creating AWS Lambda client
2025-04-22 15:00:43,482 [INFO] aws_lambda.py:97 -- AWS Lambda client created - Region: us-west-2
2025-04-22 15:00:43,483 [INFO] aws_lambda.py:450 -- Deploying runtime: virtualizarr-runtime - Memory: 1000 Timeout: 300
2025-04-22 15:01:10,015 [DEBUG] aws_lambda.py:155 -- "lithops-worker-r2ie-361dev0-a857601c4a" function is being deployed... (status: Pe

In [15]:
%%time
combined_vds_single_simulation = vz.open_virtual_mfdataset(
    single_simulation_urls[0:2],
    combine="by_coords",
    coords="minimal", compat="override",
    loadable_variables=LOW_DIMENSIONAL_VARS,
    decode_times=False,
    reader_options={'storage_options': s3fs_kwargs},
    parallel='lithops',
)
combined_vds_single_simulation

2025-04-22 15:06:32,389 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-04-22 15:06:32,491 [INFO] aws_s3.py:59 -- S3 client created - Region: us-west-2
2025-04-22 15:06:33,064 [INFO] aws_lambda.py:97 -- AWS Lambda client created - Region: us-west-2
2025-04-22 15:06:33,066 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-04-22 15:06:33,119 [INFO] aws_s3.py:59 -- S3 client created - Region: us-west-2
2025-04-22 15:06:33,576 [INFO] aws_lambda.py:97 -- AWS Lambda client created - Region: us-west-2
2025-04-22 15:06:33,578 [INFO] invokers.py:119 -- ExecutorID 78db0b-1 | JobID M000 - Selected Runtime: virtualizarr-runtime - 1000MB
2025-04-22 15:06:33,767 [INFO] invokers.py:186 -- ExecutorID 78db0b-1 | JobID M000 - Starting function invocation: _open() - Total: 2 activations
2025-04-22 15:06:33,769 [INFO] invokers.py:225 -- ExecutorID 78db0b-1 | JobID M000 - View execution logs at /private/var/folders/6x/yyxxlhtd3db164lh5zgfhgp80000gn/T/lithops-tom/logs/78db0b-1-M000

    0%|          | 0/2  

2025-04-22 15:07:00,917 [WARNING] future.py:254 -- ExecutorID 78db0b-1 | JobID M000 - CallID: 00000 - There was an exception - Activation ID: ae10bc3e-caa3-4e01-b61c-fe6554a6c545 - KeyError
2025-04-22 15:07:00,920 [WARNING] future.py:254 -- ExecutorID 78db0b-1 | JobID M000 - CallID: 00001 - There was an exception - Activation ID: ff5731b1-52b0-428b-aee0-fd595ab53c70 - KeyError
2025-04-22 15:07:00,921 [INFO] executors.py:618 -- ExecutorID 78db0b-1 - Cleaning temporary data


KeyError: 'x-amz-bucket-region'

In [ ]:
%%time
combined_vds_single_simulation = vz.open_virtual_mfdataset(
    single_simulation_urls[0:1],
    combine="by_coords",
    coords="minimal", compat="override",
    loadable_variables=LOW_DIMENSIONAL_VARS,
    decode_times=False,
    reader_options={'storage_options': s3fs_kwargs},
    parallel='lithops',
)
combined_vds_single_simulation